In [1]:
import torch
import torch.nn as nn

In [2]:
config = [
    (32, 3, 1),
    (64, 3, 2),
    ["B", 1],
    (128, 3, 2),
    ["B", 2],
    (256, 3, 2),
    ["B", 8],
    (512, 3, 2),
    ["B", 8],
    (1024, 3, 2),
    ["B", 4],  # To this point is Darknet-53
    (512, 1, 1),
    (1024, 3, 1),
    "S",
    (256, 1, 1),
    "U",
    (256, 1, 1),
    (512, 3, 1),
    "S",
    (128, 1, 1),
    "U",
    (128, 1, 1),
    (256, 3, 1),
    "S",
]

In [6]:
class CNNBlock(nn.Module):
    def __init(self, in_channels, out_channels, bn_act=True, **kwargs):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=not bn_act, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels)
        self.leaky_relu = nn.LeakyReLU(0.1)
        self.use_bn_act = bn_act
        
    def forward(self, x):
        if self.use_bn_act:
            return self.leaky_relu(self.bn(self.conv(x)))
        else:
            return self.conv(x)

In [7]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, use_residual = True, num_repeat = 1):
        super().__init__()
        self.layers = nn.ModuleList()
        for repeat in range(num_repeat):
            self.layers += [
                nn.Sequential(
                    CNNBlock(channels, channels // 2, kernel_size=1),
                    CNNBlock(channels // 2, channels, kernel_size=3, padding=1),)
            ]
        self.use_residual = use_residual
        self.num_repeats = num_repeat
        
    def forward(self, x):
        for layer in self.layers:
            if self.use_residual:
                x = layer(x) + x
            else:
                x = layer(x)
                
        return x

In [5]:
class ScalePrediction(nn.Module):
    def __init__(self. in_channels, num_classes):
        super().__init__()
        self.pred = nn.Sequential(
        CNNBlock(in_channels, 2*in_channels, kernel_size = 3, padding=1)
        CNNBlock(2*in_channels, (num_classes+5)*3, bn_act=False, kerne_size=1)
        )
        self.num_classes = num_classes
        
    def forward(self, x):
        return (self.pred(x).reshape(x.shape[0], self.num_classes + 5, x.shape[2], x.shape[3])
               .permute(0, 1, 3, 4, 2))



In [16]:
class YoloV3(nn.Module):
    def __init__(self, in_channels = 3, num_classes=20):
        super().__init__()
        self.num_classes = num_classes
        self.in_channels = in_channels
        self.layers = self._create_conv_layers()
        
    def forward(self, x):
        outputs = []
        route_connection = []
        for layer in self.layers:
            if isinstance(layer, ScalePrediction):
                outputs.append(layer(x))
                continue
                
            x = layer(x)
            
            if isinstance(layer, ResidualBlock) and layer.num_repeats==8:
                route_connection.append(x)
            elif isinstance(layer, nn.Upsample):
                x = torch.cat([x, route_connection[-1]], dim=1)
                route_connection.pop()
    
    def _create_conv_layers(self):
        layers = nn.ModuleList()
        in_channels = self.in_channels
        
        for item in config:
            
            if isinstance(item, tuple):
                out_channels, kernel_size, stride = item
                layers.append(
                CNNBlock(in_channels, out_channels, kernel_size=kernel_size, stride = stride, 
                         padding=1 if kernel_size==3 else 0)
                )
                in_channels = out_channels
            
            elif isinstance(item, list):
                
                num_repeats = item[1]
                layers.append(ResidualBlock(in_channels, num_repeat=num_repeats))
                
            elif isinstance(item, str):
                if item == "S":
                    layers += [
                        ResidualBlock(in_channels, use_residual=False, num_repeat=1),\
                        CNNBlock(in_channels, in_channels//2, kernel_size=1),
                        ScalePrediction(in_channels//2, num_classes=self.num_classes)
                    ]
                    in_channels = in_channels//2
                elif item=="U":
                    layers.append(nn.Upsample(scale_factor=2))
                    in_channels = in_channels*3
                    
            return layers
        
        

In [17]:
if __name__ == "__main__":
    num_classes = 20
    IMAGE_SIZE = 416
    model = YoloV3(num_classes=num_classes)
    x = torch.randn((2, 3, IMAGE_SIZE, IMAGE_SIZE))
    out = model(x)
    assert model(x)[0].shape == (2, 3, IMAGE_SIZE//32, IMAGE_SIZE//32, num_classes + 5)
    assert model(x)[1].shape == (2, 3, IMAGE_SIZE//16, IMAGE_SIZE//16, num_classes + 5)
    assert model(x)[2].shape == (2, 3, IMAGE_SIZE//8, IMAGE_SIZE//8, num_classes + 5)
    print("Success!")

TypeError: __init__() got an unexpected keyword argument 'kernel_size'